<center><img src="https://em-content.zobj.net/thumbs/240/apple/325/artist_1f9d1-200d-1f3a8.png" width="120">
</center>

### <center>Use this notebook to run your text-to-image server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>

## Install dependencies

In [1]:
!git clone https://github.com/saharmor/dalle-playground.git &> /dev/null
!pip install -r dalle-playground/backend/requirements.txt &> /dev/null
!pip install diffusers["torch"] &> /dev/null
!pip install jax==0.3.25 jaxlib==0.3.25 flax==0.6.2 &> /dev/null

# Run the backend web server

In [2]:
from threading import Thread, Event

app_port = 8000
announce_url = None
cloudflared_startup = Event()

def update_announce_url(url):
    global announce_url
    announce_url = url

def start_cloudflared(port):
  from flask_cloudflared import _run_cloudflared
  try:
    announce_url = _run_cloudflared(port, 8888)
  except:
    raise
  finally:
    update_announce_url(announce_url)
    cloudflared_startup.set()

def run_with_cloudflared(thread):
    old_run = thread.run

    def new_run(*args, **kwargs):
        new_thread = Thread(target = start_cloudflared, args=(app_port, ))
        new_thread.setDaemon(True)
        new_thread.start()
        old_run(*args, **kwargs)

    thread.run = new_run

def app():
  !python dalle-playground/backend/app.py --port {app_port} --save_to_disk true --img_format jpeg --output_dir generations

if __name__ == '__main__':
    t1 = Thread(target = app)
    run_with_cloudflared(t1)
    t1.start()
    cloudflared_startup.wait()
    print(f"Your url is: https://saharmor.github.io/dalle-playground/?backendUrl={announce_url}")
    t1.join()

<ipython-input-2-cf853ccabe29>:26: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  new_thread.setDaemon(True)


 * Downloading cloudflared for Linux x86_64...
Your url is: https://saharmor.github.io/dalle-playground/?backendUrl=https://leading-chance-street-geek.trycloudflare.com
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2023-09-25 11:11:39.659162: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
--> Starting the image generation server. This might take up to two minutes.
unet/diffusion_pytorch_model.safetensors not found
Fetching 12 files:   0% 0/12 [00:00<?, ?it/s]






































































Fetching 12 files:  33% 4/12 [00:06<00:15,  1.97s/it]
Fetching 12 files: 100% 12/12 [00:11<00:00,  1.07it/s]
Loading pipeline components...: 100% 5/5 [00:03<00:00,  1.50it/s]
--> Image generation server is up and

KeyboardInterrupt: ignored